# Data cleaning

In [1]:
#imports
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import re

from sklearn.impute import KNNImputer

In [2]:
#import dataset

df = pd.read_excel("data/dataset (1).xlsx")

In [3]:
dfs = {}
for i in range(1, 14):
    if i not in [5, 8, 12]:
        df = pd.read_excel(f"data/dataset ({i}).xlsx")
        dfs[f"macallan_{i}"] = df[df["Distillery"] == "Macallan"]
        dfs[f"yamazaki_{i}"] = df[df["Distillery"] == "Yamazaki"]
        dfs[f"hibiki_{i}"] = df[df["Distillery"] == "Hibiki"]

In [4]:
def concatter(dict_, name):
    #concat all files
    new_df = pd.DataFrame()
    for key, value in dfs.items():
        if name in key:
            new_df = pd.concat([new_df, value], axis = 0)
        else:
            pass
    return new_df

#concat loop

In [5]:
macallan = concatter(dfs, "macallan")
yamazaki = concatter(dfs, "yamazaki")
hibiki = concatter(dfs, "hibiki")

# Data Cleaning

In [6]:
new_dfs = {"macallan": macallan, "hibiki": hibiki, "yamazaki": yamazaki}

def volume_parser(string):
    x = string.replace("cl", "")
    x = float(x)
    return x


def percentage_converter(value):
    
    '''converts percentage data into float'''
    value = value.replace("%", "")
    return float(value)

def datacleaner(dfs, torename):
    new_dict = {}
    
    '''deals with common formatting issues in the dataframe, returns formatted df'''
    for name, df in dfs.items():

        for column in df.columns:
            if "Unnamed" in column:
                df.drop(columns = column, inplace = True)
        try:
            df.drop(columns = "No", inplace = True)
        except:
            pass

        for column in df.columns:
            try:
                df[column] = df[column].replace(r"\n", "", regex = True)
    #replace anything that isn’t alphanumeric
                #df[column] = df[column].replace(r'[^A-Za-z0-9 ]+', "", regex = True)
                df[column] = df[column].str.strip()
            except AttributeError:
                pass

        #percentage conversion to float
        #df["Bottled Strength"] = df["Bottled Strength"].apply(percentage_converter)

        df.columns = df.columns.str.lower()
        df.columns = df.columns.str.replace(" ", "_")
        df.rename(columns = torename, inplace = True)    
        df = df.reindex(sorted(df.columns), axis=1)
        new_dict[name] = df
        

    return new_dict

In [7]:
new_dfs = datacleaner(new_dfs, torename = {"bottle_size": "volume", "bottled_strength": "abv"})

# Converting string data to int/float

Converting abv and volume into float.

In [8]:
def replacer(dfs):
    new = {}
    for name, df in dfs.items():
        for i in range(len(df)):
            try:
                df["abv"].iloc[i] = percentage_converter(df["abv"].iloc[i])
            except:
                pass
        new[name] = df
    
    for name, df in dfs.items():
        for i in range(len(df)):
            try:
                df["volume"].iloc[i] = volume_parser(df["volume"].iloc[i])
            except:
                pass
        new[name] = df
    
    for name, df in dfs.items():
        df["res"] = df["res"].map({"Reserve not met": 0, "Reserve has been met": 1})
    
    return new

In [9]:
new_dfs = replacer(new_dfs)

C:\Users\ordersofmagnitude\Anaconda\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


Converting age and bid into float.

In [10]:

def value_extracter(dfs):
    new = {}
    for name, df in dfs.items():
        for i in range(len(df)):
            try:
                match = re.findall('[0-9]+', df["age"].iloc[i])
                df["age"].iloc[i] = int(match[0])
            except:
                pass
        for i in range(len(df)):
            try:
                match = re.findall('[0-9]+', df["bid"].iloc[i])
                df["bid"].iloc[i] = int(match[0])
            except:
                pass
        new[name] = df        
    return new

In [11]:
whiskeys = value_extracter(new_dfs)

In [12]:
def datetime_parser(dfs):
    new = {}
    for name, df in dfs.items():
        df["date"].replace("Closed: Monday 04 Jul 2022", "04/07/2022", inplace = True)
        new[name] = df
    return newrffffffffffffffffffffffffffff

In [13]:
whiskeys = datetime_parser(whiskeys)

# Null value imputation

In [16]:
#use replace to clean data that's hard to clean

whiskeys["macallan"].volume.replace("26 2/3 fl oz", "788", inplace = True)
whiskeys["macallan"].volume.replace("26 2/3 Fl Oz", "788", inplace = True)

In [18]:
whiskeys["macallan"].replace("see lot description", np.nan, inplace = True)
whiskeys["yamazaki"].replace("See lot description", np.nan, inplace = True)

In [26]:
whiskeys["macallan"].age.replace("80 proof", "80", inplace = True)

In [27]:
whiskeys["macallan"].drop(index = [77, 83, 114, 126, 135, 147])

,abv,age,bid,bottler,cask_type,date,distillery,distillery_status,lot,name,price_gbp,region,res,vintage,volume
0,53.4,67,19.0,Official,1st Fill Sherry,04/07/2022,Macallan,Operational,5084012,Macallan 1950 Exceptional Cask #1683-13 / 2018...,41500.0,Speysdie,0.0,1950,70.0
4,59.2,39,27.0,Official,remade refill cask,04/07/2022,Macallan,Operational,6141947,Macallan 1978 Fine and Rare 39 Year Old #13810,27130.0,Speyside,1.0,1978,70.0
8,40.0,52,19.0,Official,NaN,04/07/2022,Macallan,Operational,5079601,Macallan 1946 Select Reserve 52 Year Old,19500.0,Speyside,1.0,1946,70.0
10,49.5,NaN,NaN,Official,6 x European & American Oak,04/07/2022,Macallan,Operational,6143620,Macallan Distil Your World New York,18000.0,Speyside,NaN,NaN,70.0
17,57.0,14,NaN,R.W. Duthie for Samaroli,Sherry Wood,04/07/2022,Macallan,Operational,6136043,Macallan 1970 Samaroli 14 Year Old / Osteria A...,13500.0,Speyside,NaN,1970,75.0
20,55.2,21,123.0,Official,1st fill Spanish oak sherry,04/07/2022,Macallan,Operational,5085075,Macallan 1989 Fine and Rare 21 Year Old #3247,13000.0,Speyside,1.0,1989,70.0
21,57.5,NaN,8.0,Offiial,First-Fill Olososo,04/07/2022,Macallan,Operational,321600,Macallan Distil Your World London,12500.0,Speyside,0.0,NaN,70.0
24,43.0,NaN,27.0,Official,NaN,04/07/2022,Macallan,Operational,6139600,Macallan 1938 Handwritten Label / Rinaldi Import,12022.0,Speyside,1.0,1938,75.0
25,65.0,NaN,56.0,official,NaN,04/07/2022,Macallan,Operational,6111306A,Macallan 2007 The Ultimate Dinner Single Cask ...,12000.0,Speyside,1.0,2007,70.0
26,52.8,13,NaN,Official,Sherry Hogshead,04/07/2022,Macallan,Operational,5085077,Macallan 1995 Easter Elchies 2009,12000.0,Speyside,NaN,1995,70.0


In [14]:
def imputer(dfs, selected_columns):
    new = {}
        
    for name, df in dfs.items():
        imputer = KNNImputer()
        tmp_df = df.filter(selected_columns)   
        df[col] = imputer.fit_transform(tmp_df) #do i need .ravel()?
        new[name] = tmp_df
    return new